In [1]:
# --- 1. Install Required Libraries ---
!pip install -qU langchain sentence-transformers transformers faiss-cpu langchain-community
!pip install -qU openai

In [2]:
# --- 2. Import Required Modules ---
import os
import faiss
import numpy as np
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from openai import OpenAI
from pprint import pprint



In [3]:
# --- 3. Setup Embeddings ---
embedding_model = HuggingFaceEmbeddings(model_name="l3cube-pune/bengali-sentence-similarity-sbert")

/tmp/ipython-input-2723695470.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="l3cube-pune/bengali-sentence-similarity-sbert")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access publi

In [4]:
from langchain.docstore.document import Document

# --- 4. Prepare Chunks with Metadata (Bangla topics) ---
shikkha_chunks = [
    Document(page_content="বাংলাদেশের শিক্ষা ব্যবস্থা প্রাথমিক, মাধ্যমিক এবং উচ্চমাধ্যমিক স্তরে বিভক্ত।", metadata={"category": "শিক্ষা", "difficulty": "basic"}),
    Document(page_content="শিক্ষক উন্নয়নের জন্য নিয়মিত প্রশিক্ষণ প্রয়োজন।", metadata={"category": "শিক্ষা", "difficulty": "intermediate"}),
    Document(page_content="শিক্ষার্থীদের নৈতিক শিক্ষা প্রদান করা জরুরি।", metadata={"category": "শিক্ষা", "difficulty": "basic"})
]

shasthya_chunks = [
    Document(page_content="স্বাস্থ্যকর জীবনযাপন জন্য সুষম খাদ্য গ্রহণ করা প্রয়োজন।", metadata={"category": "স্বাস্থ্য", "difficulty": "basic"}),
    Document(page_content="নিয়মিত ব্যায়াম শরীরের জন্য উপকারী।", metadata={"category": "স্বাস্থ্য", "difficulty": "basic"}),
    Document(page_content="ডায়াবেটিস রোগ নিয়ন্ত্রণে খাদ্যাভ্যাস এবং ওষুধের নিয়ম মেনে চলা জরুরি।", metadata={"category": "স্বাস্থ্য", "difficulty": "intermediate"})
]

bhraman_chunks = [
    Document(page_content="বাংলাদেশের সুন্দরবন ভ্রমণের জন্য বিখ্যাত।", metadata={"category": "ভ্রমণ", "difficulty": "basic"}),
    Document(page_content="পর্যটকদের জন্য নৌকা ভ্রমণ একটি আকর্ষণীয় অভিজ্ঞতা।", metadata={"category": "ভ্রমণ", "difficulty": "basic"}),
    Document(page_content="ঐতিহাসিক স্থানগুলোতে দর্শনার্থীদের নিয়মিত গাইড নেওয়া উচিত।", metadata={"category": "ভ্রমণ", "difficulty": "intermediate"})
]

projukti_chunks = [
    Document(page_content="কম্পিউটার প্রযুক্তি আধুনিক শিক্ষায় গুরুত্বপূর্ণ ভূমিকা রাখে।", metadata={"category": "প্রযুক্তি", "difficulty": "basic"}),
    Document(page_content="ইন্টারনেট নিরাপত্তা নিশ্চিত করতে পাসওয়ার্ড ব্যবস্থাপনা গুরুত্বপূর্ণ।", metadata={"category": "প্রযুক্তি", "difficulty": "intermediate"}),
    Document(page_content="মোবাইল অ্যাপ্লিকেশন উন্নয়নের জন্য প্রোগ্রামিং দক্ষতা জরুরি।", metadata={"category": "প্রযুক্তি", "difficulty": "intermediate"})
]

kheladhula_chunks = [
    Document(page_content="ফুটবল বাংলাদেশের জনপ্রিয় খেলা।", metadata={"category": "খেলাধুলা", "difficulty": "basic"}),
    Document(page_content="ক্রিকেটে নিয়মিত অনুশীলন দক্ষতা বাড়ায়।", metadata={"category": "খেলাধুলা", "difficulty": "basic"}),
    Document(page_content="অলিম্পিক খেলার জন্য ক্রীড়াবিদদের কড়া প্রস্তুতি প্রয়োজন।", metadata={"category": "খেলাধুলা", "difficulty": "advanced"})
]

# Combine all chunks into a single list for vector store
all_chunks = shikkha_chunks + shasthya_chunks + bhraman_chunks + projukti_chunks + kheladhula_chunks


In [5]:
vector_store = FAISS.from_documents(all_chunks, embedding_model)

In [6]:
# --- 7. Setup GitHub-hosted OpenAI LLM (via OpenAI SDK) ---
from google.colab import userdata
token = userdata.get('4.1nano')
# token = os.environ['GITHUB_TOKEN']
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

In [7]:
def detect_category_llm(question, client):
    """
    Detect the category/topic of a Bangla question using an LLM.
    """

    # Define Bangla topics
    topics = ["শিক্ষা", "স্বাস্থ্য", "ভ্রমণ", "প্রযুক্তি", "খেলাধুলা"]
    system_msg = f"""
    তুমি একটি শ্রেণিবিন্যাসকারী এজেন্ট। নিচের প্রশ্নটি পড়ে বলো এটি কোন ক্যাটাগরির মধ্যে পড়ে: {', '.join(topics)}.
    শুধুমাত্র একটি শব্দে বাংলায় উত্তর দাও (উদাহরণ: 'শিক্ষা')।
    """

    # Send to LLM
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": question}
        ],
        model=model,
        temperature=0,
        top_p=1.0
    )

    # Extract category
    category = response.choices[0].message.content.strip()
    # if category not in topics:
    #     category = "অজানা"  # fallback if LLM returns unexpected value

    # pprint(f"{response}")

    print(f"[LLM Router] Question: {question}")
    print(f"[LLM Router] Category detected: {category}")

    return category


In [8]:
user_question = "ডায়াবেটিস নিয়ন্ত্রণের জন্য কী খাবার এবং ব্যায়াম দরকার?"
detect_category_llm(user_question, client)

[LLM Router] Question: ডায়াবেটিস নিয়ন্ত্রণের জন্য কী খাবার এবং ব্যায়াম দরকার?
[LLM Router] Category detected: স্বাস্থ্য


'স্বাস্থ্য'

In [9]:
def detect_difficulty_llm(question, client):
    """
    Detect the difficulty level of a Bangla question using an LLM.
    """

    difficulty_levels = ["basic", "intermediate", "advanced"]

    system_msg = f"""
    তুমি একটি শ্রেণিবিন্যাসকারী এজেন্ট। নিচের প্রশ্নটি পড়ে বলো এটি কোন difficulty level-এ পড়ে: {', '.join(difficulty_levels)}.
    শুধুমাত্র একটি শব্দে difficulty level লিখো। (উদাহরণ: 'basic').
    """

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": question}
        ],
        model=model,
        temperature=0,
        top_p=1.0
    )

    difficulty = response.choices[0].message.content.strip().lower()
    # if difficulty not in difficulty_levels:
    #     difficulty = "Unknown" # fallback if LLM returns unexpected value

    print(f"[LLM Difficulty] Question: {question}")
    print(f"[LLM Difficulty] Detected: {difficulty}")

    return difficulty


In [10]:
user_question = "ডায়াবেটিস নিয়ন্ত্রণের জন্য কী খাবার এবং ব্যায়াম দরকার?"
detect_difficulty_llm(user_question, client)

[LLM Difficulty] Question: ডায়াবেটিস নিয়ন্ত্রণের জন্য কী খাবার এবং ব্যায়াম দরকার?
[LLM Difficulty] Detected: basic


'basic'

In [11]:
def filter_by_metadata(query, category, difficulty=None):
    """
    Filter vector store by category and difficulty metadata, then perform similarity search.
    """
    print("\n[Metadata Filtering] Category:", category, "Difficulty:", difficulty)

    # Filter documents
    filtered_docs = [
        doc for doc in all_chunks
        if doc.metadata['category'] == category or doc.metadata.get('difficulty') == difficulty
    ]

    if not filtered_docs:
        print("⚠️ No documents found for this filter!")
        return []

    # Temporary FAISS store with filtered docs
    temp_vector_store = FAISS.from_documents(filtered_docs, embedding_model)

    # Retrieve top 3 similar docs
    similar_docs = temp_vector_store.similarity_search(query, k=3)

    print("\n[Similarity Search Results for Query]", query)
    for doc in similar_docs:
        print(" >>", doc.page_content)

    return similar_docs


In [12]:
def ask_faq_bot(user_question: str, client):
    """
    Full RAG function: detects category + difficulty, retrieves filtered chunks,
    and generates answer using LLM.
    """

    # 1️⃣ Detect category
    category = detect_category_llm(user_question, client)

    # 2️⃣ Detect difficulty
    difficulty = detect_difficulty_llm(user_question, client)

    # 3️⃣ Metadata-filtered retrieval
    docs = filter_by_metadata(user_question, category, difficulty)
    context = "\n".join([doc.page_content for doc in docs])

    # 4️⃣ Prepare system prompt with fallback
    system_prompt = (
        f"তুমি একজন সহায়ক বাংলা সহকারী। শুধুমাত্র নিচের প্রাসঙ্গিক তথ্য থেকে উত্তর দাও। যদি প্রশ্নের উত্তর এতে না থাকে, বলো 'দুঃখিত, এই বিষয়ে আমার জানা নেই। You must not answer the question if the answer is not present in the context. Context:{context}"
    )

    # 5️⃣ Generate answer
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_question}
        ],
        model=model,
        temperature=0.1,
        top_p=0.7
    )

    answer = response.choices[0].message.content

    # 6️⃣ Return full debug info and answer
    result = {
        "user_question": user_question,
        "category_detected": category,
        "difficulty_detected": difficulty,
        "retrieved_docs": [doc.page_content for doc in docs],
        "answer": answer
    }

    return result

In [13]:
user_question = "ডায়াবেটিস নিয়ন্ত্রণের জন্য কী ব্যায়াম দরকার?"

In [14]:
category = detect_category_llm(user_question, client)          # e.g., 'শিক্ষা'
difficulty = detect_difficulty_llm(user_question, client)     # e.g., 'basic'

docs = filter_by_metadata(user_question, category, difficulty)

[LLM Router] Question: ডায়াবেটিস নিয়ন্ত্রণের জন্য কী ব্যায়াম দরকার?
[LLM Router] Category detected: স্বাস্থ্য
[LLM Difficulty] Question: ডায়াবেটিস নিয়ন্ত্রণের জন্য কী ব্যায়াম দরকার?
[LLM Difficulty] Detected: basic

[Metadata Filtering] Category: স্বাস্থ্য Difficulty: basic

[Similarity Search Results for Query] ডায়াবেটিস নিয়ন্ত্রণের জন্য কী ব্যায়াম দরকার?
 >> ডায়াবেটিস রোগ নিয়ন্ত্রণে খাদ্যাভ্যাস এবং ওষুধের নিয়ম মেনে চলা জরুরি।
 >> নিয়মিত ব্যায়াম শরীরের জন্য উপকারী।
 >> ক্রিকেটে নিয়মিত অনুশীলন দক্ষতা বাড়ায়।


In [15]:
result = ask_faq_bot(user_question, client)

print("\n[User Question]", result["user_question"])
print("[Category]", result["category_detected"])
print("[Difficulty]", result["difficulty_detected"])
print("[Retrieved Docs]")
for doc in result["retrieved_docs"]:
    print(" >>", doc)
print("\n[Bot Answer]", result["answer"])

[LLM Router] Question: ডায়াবেটিস নিয়ন্ত্রণের জন্য কী ব্যায়াম দরকার?
[LLM Router] Category detected: স্বাস্থ্য
[LLM Difficulty] Question: ডায়াবেটিস নিয়ন্ত্রণের জন্য কী ব্যায়াম দরকার?
[LLM Difficulty] Detected: basic

[Metadata Filtering] Category: স্বাস্থ্য Difficulty: basic

[Similarity Search Results for Query] ডায়াবেটিস নিয়ন্ত্রণের জন্য কী ব্যায়াম দরকার?
 >> ডায়াবেটিস রোগ নিয়ন্ত্রণে খাদ্যাভ্যাস এবং ওষুধের নিয়ম মেনে চলা জরুরি।
 >> নিয়মিত ব্যায়াম শরীরের জন্য উপকারী।
 >> ক্রিকেটে নিয়মিত অনুশীলন দক্ষতা বাড়ায়।

[User Question] ডায়াবেটিস নিয়ন্ত্রণের জন্য কী ব্যায়াম দরকার?
[Category] স্বাস্থ্য
[Difficulty] basic
[Retrieved Docs]
 >> ডায়াবেটিস রোগ নিয়ন্ত্রণে খাদ্যাভ্যাস এবং ওষুধের নিয়ম মেনে চলা জরুরি।
 >> নিয়মিত ব্যায়াম শরীরের জন্য উপকারী।
 >> ক্রিকেটে নিয়মিত অনুশীলন দক্ষতা বাড়ায়।

[Bot Answer] দুঃখিত, এই বিষয়ে আমার জানা নেই।
